In [3]:
# Importing initial libraries
import requests
import pandas as pd
import numpy as np

url = input('Enter Wikipedia url for list of S&P500 Companies_')
# Scrapping for List of stocks in S&P500  
wiki_data = pd.read_html(url)
print(wiki_data)

Enter Wikipedia url for list of S&P500 Companies_ https://en.wikipedia.org/wiki/List_of_S%26P_500_companies


[    Symbol              Security             GICS Sector  \
0      MMM                    3M             Industrials   
1      AOS           A. O. Smith             Industrials   
2      ABT                Abbott             Health Care   
3     ABBV                AbbVie             Health Care   
4      ACN             Accenture  Information Technology   
..     ...                   ...                     ...   
498    YUM           Yum! Brands  Consumer Discretionary   
499   ZBRA    Zebra Technologies  Information Technology   
500    ZBH         Zimmer Biomet             Health Care   
501   ZION  Zions Bancorporation              Financials   
502    ZTS                Zoetis             Health Care   

                      GICS Sub-Industry    Headquarters Location  Date added  \
0              Industrial Conglomerates    Saint Paul, Minnesota  1957-03-04   
1                     Building Products     Milwaukee, Wisconsin  2017-07-26   
2                 Health Care Equipmen

In [4]:
# Extracting Stock Price Historical Data from Yahoo! Finance using Tickers from Wikipedia
import yfinance as yf
from pandas.tseries.offsets import DateOffset
from datetime import datetime

ticker_df = wiki_data[0]
display(ticker_df.head(2))
display(ticker_df.tail(2))

def extract_stock_data():
    ticker_list= ticker_df['Symbol'].to_list()
    stock_list = ticker_df['Security'].to_list()
    print(stock_list)
    stock_name = input('Enter Stock_Name from the above displayed list for Analysis:')
    start_date = input('Enter Start Date for Historical Data in yyyy-mm-dd:')
    today = datetime.today().strftime('%Y-%m-%d')    
    row_index = ticker_df.query(f"Security == '{stock_name}'").index[0]
    stock_ticker = ticker_list[row_index]
    stock_df = yf.download(stock_ticker,start_date,today)
    stock_df.index = pd.to_datetime(stock_df.index)
    print(f"{stock_ticker}:{stock_name}")
    return(stock_df)

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916


,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873
502,ZTS,Zoetis,Health Care,Pharmaceuticals,"Parsippany, New Jersey",2013-06-21,1555280,1952


In [5]:
# Calling Specific Company stock data 
# using extract_stock_data() function

df = extract_stock_data()

display(df.head(3))
display(df.tail(3))

['3M', 'A. O. Smith', 'Abbott', 'AbbVie', 'Accenture', 'Adobe Inc.', 'Advanced Micro Devices', 'AES Corporation', 'Aflac', 'Agilent Technologies', 'Air Products and Chemicals', 'Airbnb', 'Akamai', 'Albemarle Corporation', 'Alexandria Real Estate Equities', 'Align Technology', 'Allegion', 'Alliant Energy', 'Allstate', 'Alphabet Inc. (Class A)', 'Alphabet Inc. (Class C)', 'Altria', 'Amazon', 'Amcor', 'Ameren', 'American Airlines Group', 'American Electric Power', 'American Express', 'American International Group', 'American Tower', 'American Water Works', 'Ameriprise Financial', 'Ametek', 'Amgen', 'Amphenol', 'Analog Devices', 'Ansys', 'Aon', 'APA Corporation', 'Apple Inc.', 'Applied Materials', 'Aptiv', 'Arch Capital Group', 'Archer-Daniels-Midland', 'Arista Networks', 'Arthur J. Gallagher & Co.', 'Assurant', 'AT&T', 'Atmos Energy', 'Autodesk', 'Automated Data Processing', 'AutoZone', 'AvalonBay Communities', 'Avery Dennison', 'Axon Enterprise', 'Baker Hughes', 'Ball Corporation', 'Bank

Enter Stock_Name from the above displayed list for Analysis: Invesco
Enter Start Date for Historical Data in yyyy-mm-dd: 2011-01-01


[*********************100%%**********************]  1 of 1 completed

IVZ:Invesco


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2011-01-03,24.240000,24.719999,24.240000,24.459999,14.750483,5164600
2011-01-04,24.389999,24.540001,24.129999,24.410000,14.720332,5388300
2011-01-05,24.200001,24.450001,24.190001,24.389999,14.708273,6230200


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-03-13,15.64,15.87,15.55,15.74,15.74,4608100
2024-03-14,15.59,15.69,15.24,15.39,15.39,4453000
2024-03-15,15.19,15.58,15.19,15.43,15.43,5834800


In [14]:
# Feature Engineering _ additionals features for ML Analysis

import pandas_ta as ta

def technical_indicators(OHLCV_df):
    # Daily Avg Price
    OHLCV_df['avg_price'] = OHLCV_df.ta.ohlc4()
    # Simple Moving Avg 20Days
    OHLCV_df['sma_20'] = OHLCV_df.ta.sma(length=20)
    # Exponential Moving Avg 50Days
    OHLCV_df['ewma_50'] = OHLCV_df.ta.ema(length=50)
    # VWAP modified to use Annual cummulative sum
    OHLCV_df['typical_price'] = (OHLCV_df['High']+OHLCV_df['Low']+OHLCV_df['Close'])/3
    OHLCV_df['vw_typical_price'] = OHLCV_df['typical_price']*OHLCV_df['Volume']
    OHLCV_df['vwap_annual'] = OHLCV_df.groupby(OHLCV_df.index.year)['vw_typical_price'].cumsum() / OHLCV_df.groupby(OHLCV_df.index.year)['Volume'].cumsum()
    # RSI Momemtum Indicator 14Days
    OHLCV_df['rsi_14'] = OHLCV_df.ta.rsi()
    # MACD Momentum Indicator
    OHLCV_df[['ewma_12', 'ewma_26', 'MACD_signal']] = OHLCV_df.ta.macd()
    # Bolinger Bands %B Indicator
    OHLCV_df[['BB_lower','BB_middle','BB_upper','BBP','BB%B']]= OHLCV_df.ta.bbands()
    # On-Balance Volume in Millions
    OHLCV_df['OBV_in_million'] = OHLCV_df.ta.obv()/1000000
    # Average Directional Index 14Days
    OHLCV_df[['adx_14','dmp_14','dmn_14']] = OHLCV_df.ta.adx()
    # Dropping trivial Columns
    OHLCV_append_df = OHLCV_df.drop(columns = ['typical_price','vw_typical_price','Volume',
                                               'BBP','dmp_14','dmn_14']).dropna()
    return(OHLCV_append_df)
   

In [15]:
df = technical_indicators(df)

df.head()

,Open,High,Low,Close,Adj Close,avg_price,sma_20,ewma_50,vwap_annual,rsi_14,ewma_12,ewma_26,MACD_signal,BB_lower,BB_middle,BB_upper,BB%B,OBV_in_million,adx_14
Date,,,,,,,,,,,,,,,,,,,
2011-03-15,24.709999,25.059999,24.219999,24.910000,15.082359,24.724999,26.3130,25.381000,25.306641,41.510917,0.041826,-0.224199,0.266025,24.615050,25.658,26.700950,0.141402,-16.6201,19.679893
2011-03-16,24.820000,24.940001,23.870001,24.090000,14.585872,24.430000,26.1925,25.330373,25.283831,36.255621,-0.106181,-0.297765,0.191584,23.927731,25.176,26.424270,0.064998,-22.2934,20.340733
2011-03-17,24.629999,24.870001,24.350000,24.680000,14.943104,24.632500,26.0595,25.304868,25.275503,41.950165,-0.173865,-0.292359,0.118494,23.781392,24.988,26.194609,0.372370,-19.0468,20.948131
2011-03-18,25.030001,25.200001,24.590000,24.850000,15.046035,24.917500,25.9310,25.287030,25.268238,43.515924,-0.211352,-0.263877,0.052525,23.933496,24.790,25.646505,0.535026,-14.3005,20.878476
2011-03-21,25.270000,25.480000,24.980000,25.450001,15.409317,25.295000,25.8360,25.293421,25.268763,48.768233,-0.190449,-0.194380,0.003930,23.921691,24.796,25.670309,0.874010,-10.3714,20.309475


In [1]:
print(stock_list)
stock_name = input('Enter Stock_Name from the above displayed list for Analysis:')
row_index = ticker_df.query(f"Security == '{stock_name}'").index[0]
#print(row_index)
stock_ticker = list(ticker_stock_dictionary.values())[row_index]
stock_df = list(stock_price_dictionary.values())[row_index]
print(f"{stock_ticker}:{stock_name}")
stock_df

NameError: name 'stock_list' is not defined

In [51]:
stock_price_dictionary.columns()



['3M',
 'A. O. Smith',
 'Abbott',
 'AbbVie',
 'Accenture',
 'Adobe Inc.',
 'Advanced Micro Devices',
 'AES Corporation',
 'Aflac',
 'Agilent Technologies',
 'Air Products and Chemicals',
 'Airbnb',
 'Akamai',
 'Albemarle Corporation',
 'Alexandria Real Estate Equities',
 'Align Technology',
 'Allegion',
 'Alliant Energy',
 'Allstate',
 'Alphabet Inc. (Class A)',
 'Alphabet Inc. (Class C)',
 'Altria',
 'Amazon',
 'Amcor',
 'Ameren',
 'American Airlines Group',
 'American Electric Power',
 'American Express',
 'American International Group',
 'American Tower',
 'American Water Works',
 'Ameriprise Financial',
 'Ametek',
 'Amgen',
 'Amphenol',
 'Analog Devices',
 'Ansys',
 'Aon',
 'APA Corporation',
 'Apple Inc.',
 'Applied Materials',
 'Aptiv',
 'Arch Capital Group',
 'Archer-Daniels-Midland',
 'Arista Networks',
 'Arthur J. Gallagher & Co.',
 'Assurant',
 'AT&T',
 'Atmos Energy',
 'Autodesk',
 'Automated Data Processing',
 'AutoZone',
 'AvalonBay Communities',
 'Avery Dennison',
 'Axon 